In [1260]:
import random as rnd
import numpy as np
from typing import Callable

In [1261]:
# definição do material genético. Os dois genes são as coordenadas no espaço R^2
class Genoma:
    def __init__(self, coord:tuple[float, float]):
        self.x = coord[0]
        self.y = coord[1]
    def setCoords(self, coord:tuple[float, float]):
        self.x = coord[0]
        self.y = coord[1]

In [1262]:
# gero uma população com "n" indivíduos aleatórios no espaço x_space[0] <= x< <= x_space[1] e y_space[0] <= y <= y_space[1]
def gerar_populacao(n:int, x_space:tuple[float, float], y_space:tuple[float, float]) -> list[Genoma]:
    return [Genoma((                                     # criarei um genoma com:
                    rnd.uniform(x_space[0], x_space[1]), # uma coordenada x aleatória
                    rnd.uniform(y_space[0], y_space[1])  # uma coordenada y aleatória
                  )) for i in range(n)]                  # n vezes.

In [1263]:
def cruz_troca(g1:Genoma, g2:Genoma) -> tuple[Genoma,Genoma]:
    G1 = Genoma((g1.x, g1.y))
    G2 = Genoma((g2.x, g2.y))
    if rnd.randint(0, 1) == 0:
        temp = G1.x
        G1.x = G2.x
        G2.x = temp
    else:
        temp = G1.y
        G1.y = G2.y
        G2.y = temp
    return (G1, G2) # o retorno são dois genomas com as coordenadas x trocadas ou coordenada y trocadas.

def cruz_aprox(g1:Genoma, g2:Genoma) -> tuple[Genoma,Genoma]:
    G1 = Genoma((g1.x, g1.y))
    G2 = Genoma((g2.x, g2.y))
    G1.x = (G2.x - G1.x)/3 + G1.x
    G2.x = (G1.x - G2.x)/3 + G2.x
    G1.y = (G2.y - G1.y)/3 + G1.y
    G2.y = (G1.y - G2.y)/3 + G2.y
    return (G1, G2) # o retorno são dois genomas que se aproximam na direção um do outro por 1/3 da distância original entre eles.

In [1264]:
def mutar(g:Genoma) -> Genoma:
    G = Genoma((g.x, g.y))
    angle = rnd.uniform(0, 2*np.pi)
    dist = rnd.uniform(0, 0.5)
    G.x += dist*np.cos(angle)
    G.y += dist*np.sin(angle)
    return G # o indivíduo pode sofre uma deflexão em suas coordenadas por um módulo de no máximo 0,5 em uma direção aleatória.

In [1265]:
# esta função selecionará um par de genomas, dando maior prioridade de escolha àqueles com o menor valor da função objetivo.
def selec_par_melhores(func: Callable[[Genoma], float], pop: list[Genoma]) -> list[Genoma]:
    weights = [func(indv) for indv in pop] # calcula a funcao objetivo para os genomas
    M = max(weights)
    m = min(weights)
    if m == M:
        return [pop[0], pop[-1]]
    weights = list(map(lambda x: (M - x)/(M - m), weights)) # mapeia os valores da função objetivo
    # para pesos, sendo o maior valor da func sendo 0 e o menor sendo 1, já que queremos que genomas com valores menores da func
    # tenham pesos maiores.
    return rnd.choices(population=pop,  # seleciona 2 indivíduos da população
                       weights=weights, # pesados pelo seu score
                       k=2)

# esta função selecionará um par de genomas, dando maior prioridade de escolha àqueles com o maior ou menor valor da função objetivo.
# Isto é uma tentativa de procurar direções alternativas.
def selec_par_extremos(func: Callable[[Genoma], float], pop: list[Genoma]) -> list[Genoma]:
    weights = [func(indv) for indv in pop] # calcula a funcao objetivo para os genomas
    M = max(weights)
    m = min(weights)
    if m == M:
        return [pop[0], pop[-1]]
    weights = list(map(lambda x: (((2*x - M - m)*(2*x - M - m))/((M - m)*(M - m))), weights)) # mapeia os valores da função objetivo
    # de maneira parabólica, sendo o maior valor da func sendo 1, o menor sendo 1, e os intermediários mais próximos de 0.
    return rnd.choices(population=pop,  # seleciona 2 indivíduos da população
                       weights=weights, # pesados pelo seu score
                       k=2)

In [1266]:
def func_objetivo(g:Genoma) -> float:
    return g.x*g.x - 2*g.x*g.y + 6*g.x + g.y*g.y - 6*g.y

In [1267]:
def alg_genetico(
		geradora_de_pop: Callable[[int, tuple[float, float], tuple[float, float]], list[Genoma]],
		n_indv:          int,
		taxa_mut:        float,
		taxa_cruz:       float,
		n_elite:         int,
		x_space:         tuple[float, float],
		y_space:         tuple[float, float],
		func_obj:        Callable[[Genoma], float],
		tol:             float,
		func_selecao:    Callable[[Callable[[Genoma], float], list[Genoma]], list[Genoma]],
		func_cruzamento: Callable[[Genoma, Genoma], tuple[Genoma, Genoma]],
		func_mutacao:    Callable[[Genoma], Genoma],
		max_iter:        int
) -> tuple[list[Genoma], int]:
	
	pop = geradora_de_pop(n_indv, x_space, y_space) # gerar uma população inicial
	pop = sorted(pop, key=lambda g: func_obj(g)) # ordenar os genomas para que aqueles com o menor valor de funcao objetivo apareçam primeiro
	prev_best = sum([func_obj(indv) for indv in pop]) + tol + tol # impedir que o algoritmo convirja na primeira iteração

	for i in range(max_iter):

		if abs(prev_best - sum([func_obj(indv) for indv in pop])) <= tol: # se a diferença entre 2 iterações seguidas for melhor que uma tolerância, parar o algoritmo.
			return (pop, i)
		
		prev_best = sum([func_obj(indv) for indv in pop])
		prox_pop = pop[:n_elite] # incluir a elite inalterada na proxima geração

		for j in range((len(pop) - n_elite + 1) // 2):
			pais = func_selecao(func_obj, pop) # selecionar dois individuos

			prob = rnd.uniform(0, 1)
			if prob <= taxa_cruz: # se o numero aleatorio for maior que a taxa de cruzamento, cruzar.
				pais[0], pais[1] = func_cruzamento(pais[0], pais[1])

			# para cada resultante, verificar se teremos mutação.
			prob = rnd.uniform(0, 1)
			if prob <= taxa_mut:
				pais[0] = func_mutacao(pais[0])
			prob = rnd.uniform(0, 1)
			if prob <= taxa_mut:
				pais[1] = func_mutacao(pais[1])
			
			prox_pop += pais # adicionar os novos indvíduos à nova população
		
		if (len(pop) - n_elite) % 2 != 0: # para o caso de o número de indivíduos que devemos gerar não ser divisível por 2, eliminar o último
			prox_pop = prox_pop[:-1]      # já que estaremos gerando 1 a mais do que o necessário neste caso.
		
		pop = prox_pop
		pop = sorted(pop, key=lambda g: func_obj(g))

	return (pop, max_iter)

In [1282]:
pop, n_iter = alg_genetico(
    gerar_populacao,
    10,
    0.7,
    0.7,
    2,
    (-10, 10),
    (-10, 10),
    func_objetivo,
    0.01,
    selec_par_extremos,
    cruz_aprox,
    mutar,
    100
)

print("n_iter:", n_iter)
for indv in pop:
    print(indv.x, indv.y, "f:", func_objetivo(indv))

n_iter: 65
-5.232338091602282 -2.2326677873109055 f: -8.999999891300739
-4.6878038373626625 -1.6882160218509643 f: -8.999999830103949
-4.746068502898788 -1.8027847801297796 f: -8.996783263897058
-5.260989579673138 -2.1951150763315064 f: -8.995660549809493
-5.131791153936987 -2.0454426064093862 f: -8.992543928339874
-4.769718688211999 -1.864806935374417 f: -8.990958225251578
-4.734462905498619 -1.8382102876875674 f: -8.989236480688938
-4.959526586288786 -1.8458793131798925 f: -8.987084297314917
-4.5585221328678065 -1.8044591503510017 f: -8.939514983431469
-5.1105031260700455 -1.3459219435844203 f: -8.415415615388882
